In [43]:
import pandas as pd
import geopandas as gpd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import ast


import json

import numpy as np

from datetime import timedelta, date


import re

#%pip install lxml
#%pip install requests
#%pip install sodapy

from lxml import html
import requests

import os

import glob

from sodapy import Socrata

### Get limit

In [44]:

#def request ():
#    page = requests.get('https://www.dallasopendata.com/Public-Safety/Police-Incidents/qv6i-rri7')
#    content = html.fromstring(page.content)

#    return content
#def parse():
#    content = request()

#    containers = content.xpath('/html/body/div[1]/div/div/header/div/div[3]')

#    for container in containers:
#        title = container.xpath('/html/body/main/div/div[1]/div/div[2]/forge-view-switcher/forge-view[1]/div/div/div/div[1]/section[1]/div/div/dl/div[1]/dd')


#        print(title)
#parse()


### Read in data

In [45]:
#Define Socrata API Client
client = Socrata("www.dallasopendata.com", None)

#NOTE: Change to the limit we defined earlier
lim = 1200000

#Read in data and convert to DataFrame
results = client.get("qv6i-rri7", limit=lim)

df = pd.DataFrame.from_records(results)

    # lim += 100000

print(len(df))

#Create empty lists of addresses, latitudes and longitudes to convert to columns later.
lats = []
longs = []
adds = []

#In the underlying raw date we have a column called 'Location1' where each cell contains a street address, latitude and Longitude. Lets extract those and add them to the list.
for i in range(0, len(df)):
    #If the cell isn't empty
    try:
        latitude = df['geocoded_column'][i]['latitude']
    except:
            latitude = np.nan
    try:
        longitude = df['geocoded_column'][i]['longitude']
    except:
        longitude = np.nan
    try:
        address = ''
        for val in ast.literal_eval(df['geocoded_column'][i]['human_address']).values():
             address = address + ' ' + val.strip()
    except:
         address = ''
        
    #Add the lat, long and address to the corresponding list.
    lats.append(latitude)
    longs.append(longitude)
    adds.append(address)

# #Create new columns from the populated lists.
df['latitude'] = lats
df['longitude'] = longs
df['address'] = adds 

df['date1'] = pd.to_datetime(df['date1'], errors='coerce')

1158925


In [46]:
df['date1'] = pd.to_datetime(df['date1'], errors='coerce')

df = (df
      .loc[((df['nibrs_code'] == '09A') | 
            (df['ucrcode'].isin([110, 120])))]
        .sort_values('date1', ascending=False)
        .drop_duplicates(subset = ['incidentnum'], keep='first')
        .reset_index(drop=True)
        .copy())

In [49]:
df.to_csv('murders_data.csv', index=False)